In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
from dotenv import load_dotenv

# Path setup logic
current_dir = os.getcwd()
if os.path.basename(current_dir) == "app":
    project_root = os.path.abspath("..")
elif "app" not in os.listdir(current_dir):
    project_root = os.path.abspath("..")
else:
    project_root = current_dir

if project_root not in sys.path:
    sys.path.insert(0, project_root)

load_dotenv()
print("✅ Auto-reload enabled. Path set.")

✅ Auto-reload enabled. Path set.


In [ ]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "rag-eval-sml"

In [3]:
def get_question_from_input(inputs: dict) -> str:
    """Extracts the user question from the Chat format or Flat format."""
    # Case 1: Chat Format (Your current JSONL)
    if "messages" in inputs:
        # Get the last message where role is 'user'
        for msg in reversed(inputs["messages"]):
            if msg["role"] == "user":
                return msg["content"]
    
    # Case 2: Flat Format (CSV style)
    return inputs.get("question") or inputs.get("Question") or inputs.get("input") or ""

def get_ground_truth(reference_outputs: dict) -> str:
    """Extracts the ground truth answer."""
    # Case 1: Chat Format (Your current JSONL uses 'message' -> 'content')
    if "message" in reference_outputs:
        return reference_outputs["message"]["content"]
    
    # Case 2: Flat Format
    return reference_outputs.get("answer") or reference_outputs.get("output") or ""

In [4]:
from os import getenv
from typing_extensions import Annotated, TypedDict
from langchain_openai import ChatOpenAI

# --- 1. Define Helper Functions (Ensure these are defined!) ---
def get_question_from_input(inputs: dict) -> str:
    """Extracts the user question from the Chat format or Flat format."""
    # Case 1: Chat Format (Your current JSONL)
    if "messages" in inputs:
        for msg in reversed(inputs["messages"]):
            if msg["role"] == "user":
                return msg["content"]
    # Case 2: Flat Format (CSV style)
    return inputs.get("question") or inputs.get("Question") or inputs.get("input") or ""

def get_ground_truth(reference_outputs: dict) -> str:
    """Extracts the ground truth answer."""
    # Case 1: Chat Format
    if "message" in reference_outputs:
        return reference_outputs["message"]["content"]
    # Case 2: Flat Format
    return reference_outputs.get("answer") or reference_outputs.get("output") or ""


# --- 2. Define the Schema & LLM ---
class CorrectnessGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    correct: Annotated[bool, ..., "True if the answer is correct, False otherwise."]

correctness_instructions = """You are a teacher grading a quiz. You will be given a QUESTION, the GROUND TRUTH (correct) ANSWER, and the STUDENT ANSWER. Here is the grade criteria to follow:
(1) Grade the student answers based ONLY on their factual accuracy relative to the ground truth answer. (2) Ensure that the student answer does not contain any conflicting statements.
(3) It is OK if the student answer contains more information than the ground truth answer, as long as it is factually accurate relative to the  ground truth answer.

Correctness:
A correctness value of True means that the student's answer meets all of the criteria.
A correctness value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

# NOTE: If this errors with "schema not supported", set strict=False
grader_llm = ChatOpenAI(
    api_key=getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    model="mistralai/devstral-2512:free"
).with_structured_output(
    CorrectnessGrade, method="json_schema", strict=False 
)


# --- 3. The Updated Evaluator Function ---
async def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    """An evaluator for RAG answer accuracy"""
    
    # ✅ FIX: Use helpers instead of direct dict access
    question = get_question_from_input(inputs)
    ground_truth = get_ground_truth(reference_outputs)
    
    # 'outputs' comes from your rag_bot, which is already flat, so this is fine:
    student_answer = outputs['answer'] 

    answers = f"""\
QUESTION: {question}
GROUND TRUTH ANSWER: {ground_truth}
STUDENT ANSWER: {student_answer}"""

    # Run evaluator
    grade = await grader_llm.ainvoke([
        {"role": "system", "content": correctness_instructions},
        {"role": "user", "content": answers}
    ])
    
    return grade["correct"]

d:\Projects\CustomRAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from os import getenv
from typing_extensions import Annotated, TypedDict
from langchain_openai import ChatOpenAI

# --- Helper Function (Required for these evaluators) ---
def get_question_from_input(inputs: dict) -> str:
    """Extracts the user question from nested Chat format or Flat format."""
    if "messages" in inputs:
        for msg in reversed(inputs["messages"]):
            if msg["role"] == "user":
                return msg["content"]
    return inputs.get("question") or inputs.get("Question") or inputs.get("input") or ""

# ---------------------------------------------------------
# 1. RELEVANCE EVALUATOR
# ---------------------------------------------------------
class RelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "Provide the score on whether the answer addresses the question"]

relevance_instructions = """You are a teacher grading a quiz. You will be given a QUESTION and a STUDENT ANSWER. Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is concise and relevant to the QUESTION
(2) Ensure the STUDENT ANSWER helps to answer the QUESTION

Relevance:
A relevance value of True means that the student's answer meets all of the criteria.
A relevance value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

relevance_llm = ChatOpenAI(
    api_key=getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    model="mistralai/devstral-2512:free"
).with_structured_output(
    RelevanceGrade, method="json_schema", strict=False 
)

async def relevance(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer helpfulness."""
    question = get_question_from_input(inputs) # <--- FIX
    student_answer = outputs['answer']
    
    answer = f"QUESTION: {question}\nSTUDENT ANSWER: {student_answer}"
    
    grade = await relevance_llm.ainvoke([
        {"role": "system", "content": relevance_instructions},
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

# ---------------------------------------------------------
# 2. GROUNDEDNESS EVALUATOR
# ---------------------------------------------------------
class GroundedGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    grounded: Annotated[bool, ..., "Provide the score on if the answer hallucinates from the documents"]

grounded_instructions = """You are a teacher grading a quiz. You will be given FACTS and a STUDENT ANSWER. Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is grounded in the FACTS. (2) Ensure the STUDENT ANSWER does not contain "hallucinated" information outside the scope of the FACTS.

Grounded:
A grounded value of True means that the student's answer meets all of the criteria.
A grounded value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

grounded_llm = ChatOpenAI(
    api_key=getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    model="mistralai/devstral-2512:free"
).with_structured_output(
    GroundedGrade, method="json_schema", strict=False
)

async def groundedness(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer groundedness."""
    # Ensure documents exist
    if "documents" not in outputs:
        return False
        
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nSTUDENT ANSWER: {outputs['answer']}"
    
    grade = await grounded_llm.ainvoke([
        {"role": "system", "content": grounded_instructions},
        {"role": "user", "content": answer}
    ])
    return grade["grounded"]

# ---------------------------------------------------------
# 3. RETRIEVAL RELEVANCE EVALUATOR
# ---------------------------------------------------------
class RetrievalRelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "True if the retrieved documents are relevant to the question, False otherwise"]

retrieval_relevance_instructions = """You are a teacher grading a quiz. You will be given a QUESTION and a set of FACTS provided by the student. Here is the grade criteria to follow:
(1) You goal is to identify FACTS that are completely unrelated to the QUESTION
(2) If the facts contain ANY keywords or semantic meaning related to the question, consider them relevant
(3) It is OK if the facts have SOME information that is unrelated to the question as long as (2) is met

Relevance:
A relevance value of True means that the FACTS contain ANY keywords or semantic meaning related to the QUESTION and are therefore relevant.
A relevance value of False means that the FACTS are completely unrelated to the QUESTION.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

retrieval_relevance_llm = ChatOpenAI(
    api_key=getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    model="openai/gpt-oss-120b:free"
).with_structured_output(RetrievalRelevanceGrade, method="json_schema", strict=False)

async def retrieval_relevance(inputs: dict, outputs: dict) -> bool:
    """An evaluator for document relevance"""
    question = get_question_from_input(inputs) # <--- FIX
    
    # Check for empty docs
    if "documents" not in outputs or not outputs["documents"]:
        return False
        
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nQUESTION: {question}"
    
    grade = await retrieval_relevance_llm.ainvoke([
        {"role": "system", "content": retrieval_relevance_instructions},
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

In [6]:
import uuid
from langchain_core.messages import HumanMessage, ToolMessage
from langgraph.checkpoint.memory import MemorySaver

# Import your actual graph builder
from app.services.graph.graph import build_rag_graph
# ❌ REMOVE THIS IMPORT: from app.services.graph.tools import UserContext

class EvalDocument:
    def __init__(self, content):
        self.page_content = content

async def rag_bot(question: str):
    # Use MemorySaver to isolate eval from your real DB
    checkpointer = MemorySaver()
    app_graph = build_rag_graph(checkpointer)
    
    thread_id = str(uuid.uuid4())
    eval_user_id = "fc047edd-8907-441b-a0a9-b94926a5d1c6"
    
    # ✅ FIX: Pass user_id inside 'configurable' dict
    config = {
        "configurable": {
            "thread_id": thread_id,
            "user_id": eval_user_id 
        }
    }
    
    # Run the Graph (No 'context' argument needed anymore)
    inputs = {"messages": [HumanMessage(content=question)]}
    final_state = await app_graph.ainvoke(inputs, config=config)
    
    # --- EXTRACT OUTPUTS ---
    messages = final_state["messages"]
    final_answer = messages[-1].content
    
    retrieved_docs = []
    for msg in messages:
        if isinstance(msg, ToolMessage):
            retrieved_docs.append(EvalDocument(msg.content))
            
    return {
        "answer": final_answer,
        "documents": retrieved_docs
    }

async def target(inputs: dict) -> dict:
    # 1. Extract the question safely
    question = get_question_from_input(inputs)
    
    if not question:
        raise ValueError(f"Could not find question in inputs: {inputs.keys()}")

    # 2. Run your bot
    return await rag_bot(question)

In [ ]:
experiment_results = await client.aevaluate(
    target,
    data=dataset_name,
    evaluators=[correctness, relevance, groundedness, retrieval_relevance], # Add others if you updated them
    experiment_prefix="rag-doc-relevance-v2",
)

print("✅ Evaluation Complete!")



View the evaluation results for experiment: 'rag-doc-relevance-v2-922046c5' at:
https://smith.langchain.com/o/2f1d8fe1-f343-4b0c-9485-a97483c40c16/datasets/b42fd3d0-942d-446f-818a-99952266f172/compare?selectedSessions=02d1ee73-c3af-4349-8034-11663f88a1dc




0it [00:00, ?it/s]Failed to initialize AsyncMilvusClient during Milvus initialization: <ConnectionConfigException: (code=1, message=Cannot create async connection: no running event loop. Please ensure you are running in an async context.)>. Async operations will be unavailable until AsyncMilvusClient is successfully created.


🔍 Tool Execution: Searching docs for User fc047edd-8907-441b-a0a9-b94926a5d1c6...


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


🔍 Tool Execution: Searching docs for User fc047edd-8907-441b-a0a9-b94926a5d1c6...


...

🔍 Tool Execution: Searching docs for User fc047edd-8907-441b-a0a9-b94926a5d1c6...🔍 Tool Execution: Searching docs for User fc047edd-8907-441b-a0a9-b94926a5d1c6...



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


--- LOOP LIMIT REACHED (3) ---


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
20it [1:22:23, 247.16s/it]


CancelledError: 

In [ ]:

df = experiment_results.to_pandas()
df.head()

✅ Key loaded: sk-or-v1... (length: 73)

📡 Connecting to https://openrouter.ai/api/v1/chat/completions...
Status Code: 404
❌ FAILED.
Error Response: {"error":{"message":"No endpoints found matching your data policy (Free model publication). Configure: https://openrouter.ai/settings/privacy","code":404}}


In [ ]:
import easyocr
reader = easyocr.Reader(['ch_sim','en']) # this needs to run only once to load the model into memory
result = reader.readtext('chinese.jpg')